In [1]:
print('ok')

ok


In [2]:
%pwd

'/Users/karthicksundar/Documents/learnings/LLM/local_llm/notebooks'

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader # dependency : pip install langchain-community pypdf
from langchain.text_splitter import RecursiveCharacterTextSplitter # dependency: pip install langchain-community
from langchain.embeddings import HuggingFaceEmbeddings 


from pinecone.grpc import PineconeGRPC as Pinecone # pip install protobuf google-api-core grpcio
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") # dependency: pip install sentence-transformers

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
def load_documents_for_vectordb(path: str):
    dir_loader = DirectoryLoader(path,
                                glob="*.pdf",
                                loader_cls=PyPDFLoader)
    documents = dir_loader.load()

    return documents

In [12]:
extracted_data = load_documents_for_vectordb(path="../data")

In [15]:
type(extracted_data)

list

In [16]:
def chunk_documents(data: list):
    chunker = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = chunker.split_documents(data)

    return text_chunks

In [17]:
text_chunks = chunk_documents(extracted_data)
print(f"The number of document chunks created are: {len(text_chunks)}")

The number of document chunks created are: 5859


In [ ]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [20]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [ ]:
# Create embedding on the text chunks and push to pinecone vector store on cloud
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

# Now create object for the pinecone vector store on cloud
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

# define retreiver using pinecone object
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)